# Voice-in-Head Intervention Demo

Interactive notebook for testing the Voice-in-Head intervention strategy.

In [ ]:
# Setup: Import and reload modules
import importlib
from IPython.display import HTML, display

from pipeline import voice_in_head_strategy
importlib.reload(voice_in_head_strategy)
from pipeline.voice_in_head_strategy import VoiceInHeadStrategy
from pipeline.rollout_generator import RolloutGenerator
from pipeline.intervention_inserter import InterventionInserter

# Initialize components
strategy = VoiceInHeadStrategy()
inserter = InterventionInserter(strategy)
rollout_generator = RolloutGenerator(max_tokens=10000)

print("✓ Components initialized")

## 1. Input Prompt and Goal

In [ ]:
# Input: Your prompt and desired intervention goal
prompt = "What are some fun things to do in London?"
goal_intervention = "Go for a day trip to Croydon."

print(f"Prompt: {prompt}")
print(f"Goal: {goal_intervention}")

## 2. Generate Initial Rollout

In [ ]:
# Generate full rollout from the prompt
print("Generating initial rollout...")
formatted_prompt = rollout_generator.format_question_prompt(prompt)
rollout = rollout_generator.generate(formatted_prompt, n=1)[0]

print("\n✓ Generated rollout:")
print(rollout)

## 3. Apply Voice-in-Head Intervention

In [ ]:
# Apply intervention strategy
print("Applying voice-in-head intervention...")
intervened_text, suggested_interventions = inserter.apply(
    rollout=rollout, 
    intervention_text=goal_intervention, 
    prompt=prompt
)

print(f"✓ Generated {len(suggested_interventions)} intervention candidates")
print(f"✓ Selected intervention and prepared for continuation")

## 4. Continue Generation

In [ ]:
# Generate continuation from the intervened text
print("Generating continuation...")
full_output = rollout_generator.generate(intervened_text, n=1)[0]

print("✓ Generated continuation")

## 5. Color-Coded Display

- **Blue**: Clipped original text
- **Green**: Generated intervention text
- **Yellow**: Final continuation

In [ ]:
# Parse out the three components
# intervened_text contains: <think>\n{clipped_text}{generated_intervention}
# full_output is the continuation after intervened_text

# Extract clipped text (everything before the generated intervention)
think_content = intervened_text.split("<think>\n")[1] if "<think>" in intervened_text else intervened_text

# Find where the original rollout was clipped
original_think = rollout.split("<think>")[1].split("</think>")[0] if "<think>" in rollout else rollout

# Find the clipped portion by comparing with original
# The clipped text is the common prefix between think_content and original_think
clipped_length = 0
for i, char in enumerate(think_content):
    if i < len(original_think) and char == original_think[i]:
        clipped_length = i + 1
    else:
        break

clipped_text = think_content[:clipped_length]
generated_intervention = think_content[clipped_length:]
continuation = full_output

# Create HTML with color-coded sections
html = f"""
<div style="font-family: monospace; white-space: pre-wrap; padding: 20px; background-color: #f5f5f5; border-radius: 8px;">
    <div style="margin-bottom: 20px;">
        <strong>Prompt:</strong> {prompt}
    </div>
    <div style="margin-bottom: 20px;">
        <strong>Goal Intervention:</strong> {goal_intervention}
    </div>
    <hr style="margin: 20px 0;">
    <div style="margin-bottom: 10px;">
        <strong>Generated Output:</strong>
    </div>
    <div style="line-height: 1.6;">
        <span style="background-color: #cce5ff; padding: 2px 4px; border-radius: 3px;">{clipped_text}</span><span style="background-color: #ccffcc; padding: 2px 4px; border-radius: 3px;">{generated_intervention}</span><span style="background-color: #ffffcc; padding: 2px 4px; border-radius: 3px;">{continuation}</span>
    </div>
    <hr style="margin: 20px 0;">
    <div style="margin-top: 20px; font-size: 0.9em;">
        <div><span style="background-color: #cce5ff; padding: 2px 8px; border-radius: 3px;">Blue</span> = Clipped original text</div>
        <div><span style="background-color: #ccffcc; padding: 2px 8px; border-radius: 3px;">Green</span> = Generated intervention</div>
        <div><span style="background-color: #ffffcc; padding: 2px 8px; border-radius: 3px;">Yellow</span> = Continuation</div>
    </div>
</div>
"""

display(HTML(html))

## Debug: View Raw Components

In [ ]:
# Optional: View the raw components for debugging
print("=== CLIPPED TEXT ===")
print(clipped_text)
print("\n=== GENERATED INTERVENTION ===")
print(generated_intervention)
print("\n=== CONTINUATION ===")
print(continuation)